In [ ]:
!pip install "C:\Users\Zerov\TA_Lib-0.4.24-cp39-cp39-win_amd64.whl"

In [ ]:
!pip install requests
!pip install numpy
!pip install python-binance


In [ ]:
api_key='******************************************************************'
api_secret='*****************************************************************'

In [1]:
from binance.client import Client
import requests
import numpy as np
import talib
import time

In [2]:
SYMBOL='ALGOUSDT'
INTERVAL='15m'
LIMIT='200'
QNTY=35

In [ ]:
client=Client(api_key, api_secret)

In [ ]:
def get_data():
    url='https://api.binance.com/api/v3/klines?symbol={}&interval={}&limit={}'.format(SYMBOL, INTERVAL, LIMIT)
    res=requests.get(url)
    return_data=[]
    for each in res.json():
        return_data.append(float(each[4]))
    return np.array(return_data)
#собираем только цены закрытия


In [8]:
url='https://api.binance.com/api/v3/klines?symbol={}&interval={}&limit={}'.format(SYMBOL, INTERVAL, LIMIT)
res=requests.get(url)
a=res.json()[0]
a

[1672663500000,
 '0.18110000',
 '0.18140000',
 '0.18090000',
 '0.18110000',
 '307192.00000000',
 1672664399999,
 '55648.89960000',
 144,
 '90531.00000000',
 '16399.69700000',
 '0']

In [ ]:
def place_order(order_type):
    if(order_type =='buy'):
        order=client.ceate_order(symbol=SYMBOL, side='buy', type='MARKET', quantity=QNTY)
        print('Open position', order)
    else:
        order=client.ceate_order(symbol=SYMBOL, side='sell', type='MARKET', quantity=QNTY)
        print('Close position', order)
    return

In [ ]:
def main():
    buy = False
    sell = True
    ema_8 = None
    emd_21 = None
    last_ema_8 = None
    last_ema_21 = None
    print(SYMBOL)
    print(INTERVAL)
    print(QNTY)
    
    while True:
        closing_data=get_data()
        ema_8=talib.EMA(closing_data, 8)[-1]
        ema_21=talib.EMA(closing_data, 21)[-1]
        
        if(ema_8 > ema_21 and last_ema_8):
            if(ema_8 < ema_21 and not buy):
                print('Buy!')
                place_order('buy')
                buy = True
                sell = False
        
        if(ema_21 > ema_8 and last_ema_21):
            if(last_ema_21<last_ema_8 and not sell):
                print('Sell!')
                place_order('sell')
                sell=True
                buy=False
        last_ema_8= ema_8
        last_ema_21= ema_21
        time.sleep(1)
         

In [ ]:
if __name__=='__main__':
    main()